In [2]:
import torch
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset, TensorDataset
from torchvision import transforms, utils
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

In [108]:
ROOT = "/StudentData/HW2/train/"
TestFolder  = "/StudentData/HW2/test/"

In [102]:
class ImageDataset(Dataset):
    def __init__(self,root,transform=None):
        super().__init__()
        self.root = root
        self.transform = transform
        self.idxtoImage = {i:file for i,file in enumerate(os.listdir(self.root))}

        
    def __len__(self):
        return len(self.idxtoImage)
    
    def __getitem__(self,idx):
        file = self.idxtoImage[idx]
        image = Image.open(self.root+file).convert("RGB")
        if self.transform !=None:
            image = self.transform(image)

        label = int(file.split('_')[1].split('.')[0])
        return image,label
        

In [113]:
# Hyper Parameters
num_epochs = 10
batch_size = 100
learning_rate = 0.001

transform = transforms.Compose([transforms.Resize((48,48)),transforms.ToTensor()])
train_dataset = ImageDataset(ROOT,transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=False)

test_dataset = ImageDataset(TestFolder,transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size, 
                                           shuffle=False)

In [104]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=2),
            nn.ReLU())
        
        self.fc1 = nn.Linear(2*2*128, 256)
        self.ReLU = nn.ReLU()
        self.fc2 = nn.Linear(256,2)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = out.view(out.size(0), -1)
  
        out = self.fc1(out)
        out = self.ReLU(out)
        out = self.fc2(out)       

        return out

In [105]:
def to_gpu(x):
    return x.cuda() if torch.cuda.is_available() else x

In [106]:
model = Model()
model = to_gpu(model)
    
# convert all the weights tensors to cuda()
# Loss and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print('number of parameters: ', sum(param.numel() for param in model.parameters()))
print(f'Num of trainable parameters : {sum(p.numel() for p in model.parameters() if p.requires_grad)}')

number of parameters:  221058
Num of trainable parameters : 221058


In [107]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = to_gpu(images)
        labels = to_gpu(labels)
        
        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1,
                     len(train_dataset)//batch_size, loss.item()))

Epoch [1/10], Iter [100/182] Loss: 0.1443
Epoch [2/10], Iter [100/182] Loss: 0.1201
Epoch [3/10], Iter [100/182] Loss: 0.1055
Epoch [4/10], Iter [100/182] Loss: 0.0883
Epoch [5/10], Iter [100/182] Loss: 0.0789
Epoch [6/10], Iter [100/182] Loss: 0.0769
Epoch [7/10], Iter [100/182] Loss: 0.0919
Epoch [8/10], Iter [100/182] Loss: 0.0764
Epoch [9/10], Iter [100/182] Loss: 0.0736
Epoch [10/10], Iter [100/182] Loss: 0.0619


In [123]:
model.eval() 
correct = 0
total = 0
i=0
for images, labels in test_loader:
    images = to_gpu(images)
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)

    total+= labels.size(0)
    correct += (predicted.cpu() == labels).sum()
    i+=1
    
print('Test Accuracy of the model on the test images: %d %%' % (100 * correct / total))

6086
Test Accuracy of the model on the test images: 96 %
